In [9]:
import pandas as pd
import numpy as np
import os
import uuid

df_list01 = []
df_list02 = []
final_df_list01 = []
final_df_list02 = []
for root, directories, files in os.walk('./DataSet'):
    for file in files:
        if file.endswith('right.csv'):
            df = pd.read_csv(os.path.join(root, file))
            df['procedure'] = 'network01'
            df_list01.append(df)
        elif file.endswith('left.csv'):
            df = pd.read_csv(os.path.join(root, file))
            df['procedure'] = 'network02'
            df_list02.append(df)
        elif file.endswith('labels.csv'):
            df = pd.read_csv(os.path.join(root, file))
            df['dataset'] = root.rsplit('/', 1)[-1]
            df_list01.append(df)
            df_list02.append(df)
    if df_list01:
         combined_df01 = pd.concat(df_list01, axis=1)
         final_df_list01.append(combined_df01)
         df_list01.clear()
    else:
         combined_df = pd.DataFrame()

    if df_list02:
         combined_df02 = pd.concat(df_list02, axis=1)
         final_df_list02.append(combined_df02)
         df_list02.clear()
    else:
         combined_df = pd.DataFrame()

final_df01 = pd.concat(final_df_list01, axis=0)
final_df02 = pd.concat(final_df_list02, axis=0)

final_df = pd.concat([final_df01, final_df02], axis=0)

new_column_names = {
    'temp_dashboard': 'temp_t_dashboard',
    'temp_above_suspension': 'temp_t_above_suspension',
    'temp_below_suspension': 'temp_t_below_suspension'
}
final_df.rename(columns=new_column_names, inplace=True)


Road_Surface_Condition = final_df[['paved_road', 'unpaved_road']]
Road_Surface_Type_Labels = final_df[['dirt_road', 'cobblestone_road', 'asphalt_road']]
Road_Roughness_Condition_Left = final_df[['good_road_left', 'regular_road_left','bad_road_left']]
Road_Roughness_Condition_Right = final_df[['good_road_right', 'regular_road_right','bad_road_right']]
Speed_Bump_Types= final_df[['no_speed_bump', 'speed_bump_asphalt','speed_bump_cobblestone']]
final_df['Road_Surface_Type_Labels'] = pd.Series(Road_Surface_Type_Labels.columns[np.where(Road_Surface_Type_Labels !=0)[1]])
final_df['Road_Surface_Condition'] = pd.Series(Road_Surface_Condition.columns[np.where(Road_Surface_Condition !=0)[1]])
final_df['Road_Roughness_Condition_Left'] = pd.Series(Road_Roughness_Condition_Left.columns[np.where(Road_Roughness_Condition_Left !=0)[1]])
final_df['Road_Roughness_Condition_Right'] = pd.Series(Road_Roughness_Condition_Right.columns[np.where(Road_Roughness_Condition_Right !=0)[1]])
final_df['Speed_Bump_Types'] = pd.Series(Speed_Bump_Types.columns[np.where(Speed_Bump_Types !=0)[1]])
final_df = final_df.drop(columns=['paved_road', 'unpaved_road','dirt_road', 'cobblestone_road', 'asphalt_road','good_road_left', 'regular_road_left','bad_road_left',                                      'good_road_right', 'regular_road_right','bad_road_right', 'no_speed_bump', 'speed_bump_asphalt','speed_bump_cobblestone'])
print(final_df.columns)

Index(['timestamp', 'acc_x_dashboard', 'acc_y_dashboard', 'acc_z_dashboard',
       'acc_x_above_suspension', 'acc_y_above_suspension',
       'acc_z_above_suspension', 'acc_x_below_suspension',
       'acc_y_below_suspension', 'acc_z_below_suspension', 'gyro_x_dashboard',
       'gyro_y_dashboard', 'gyro_z_dashboard', 'gyro_x_above_suspension',
       'gyro_y_above_suspension', 'gyro_z_above_suspension',
       'gyro_x_below_suspension', 'gyro_y_below_suspension',
       'gyro_z_below_suspension', 'mag_x_dashboard', 'mag_y_dashboard',
       'mag_z_dashboard', 'mag_x_above_suspension', 'mag_y_above_suspension',
       'mag_z_above_suspension', 'temp_t_dashboard', 'temp_t_above_suspension',
       'temp_t_below_suspension', 'timestamp_gps', 'latitude', 'longitude',
       'speed', 'procedure', 'dataset', 'Road_Surface_Type_Labels',
       'Road_Surface_Condition', 'Road_Roughness_Condition_Left',
       'Road_Roughness_Condition_Right', 'Speed_Bump_Types'],
      dtype='object')


In [12]:
data_collection_contexts = pd.read_csv('./DataSet/data_collection_contexts.csv')
PVS_Dataset_complete  = final_df.set_index('dataset').join(data_collection_contexts.set_index('Dataset'))
shuffled_df = PVS_Dataset_complete.sample(frac=1, random_state=42)
PVS_Dataset = shuffled_df[:1000]
print(PVS_Dataset.columns)

Index(['timestamp', 'acc_x_dashboard', 'acc_y_dashboard', 'acc_z_dashboard',
       'acc_x_above_suspension', 'acc_y_above_suspension',
       'acc_z_above_suspension', 'acc_x_below_suspension',
       'acc_y_below_suspension', 'acc_z_below_suspension', 'gyro_x_dashboard',
       'gyro_y_dashboard', 'gyro_z_dashboard', 'gyro_x_above_suspension',
       'gyro_y_above_suspension', 'gyro_z_above_suspension',
       'gyro_x_below_suspension', 'gyro_y_below_suspension',
       'gyro_z_below_suspension', 'mag_x_dashboard', 'mag_y_dashboard',
       'mag_z_dashboard', 'mag_x_above_suspension', 'mag_y_above_suspension',
       'mag_z_above_suspension', 'temp_t_dashboard', 'temp_t_above_suspension',
       'temp_t_below_suspension', 'timestamp_gps', 'latitude', 'longitude',
       'speed', 'procedure', 'Road_Surface_Type_Labels',
       'Road_Surface_Condition', 'Road_Roughness_Condition_Left',
       'Road_Roughness_Condition_Right', 'Speed_Bump_Types', 'Vehicle',
       'Driver', 'Scenario', 

In [15]:
property = ['acc_x_dashboard', 'acc_y_dashboard', 'acc_z_dashboard',
       'acc_x_above_suspension', 'acc_y_above_suspension',
       'acc_z_above_suspension', 'acc_x_below_suspension',
       'acc_y_below_suspension', 'acc_z_below_suspension', 'gyro_x_dashboard',
       'gyro_y_dashboard', 'gyro_z_dashboard', 'gyro_x_above_suspension',
       'gyro_y_above_suspension', 'gyro_z_above_suspension',
       'gyro_x_below_suspension', 'gyro_y_below_suspension',
       'gyro_z_below_suspension', 'mag_x_dashboard', 'mag_y_dashboard',
       'mag_z_dashboard', 'mag_x_above_suspension', 'mag_y_above_suspension',
       'mag_z_above_suspension', 'temp_t_dashboard', 'temp_t_above_suspension',
       'temp_t_below_suspension']

labels = []
table_list = []
for column_name in PVS_Dataset.columns:
    if column_name in property:
        column_values = PVS_Dataset[column_name].values.tolist()

        new_df = pd.DataFrame({'measure': column_values})

        column_name_split = column_name.split('_')
        if len(column_name_split) == 3 :
            new_df['sensor'] = column_name_split[0]
            new_df['property'] = '_'.join(column_name_split[:2])
            new_df['feature'] = column_name_split[-1]
            if column_name_split[0] == "mag":
                new_df['platform'] = "AK8963"
            else:
                new_df['platform'] = 'MPU-6500'
        elif len(column_name_split) == 4 :
            new_df['sensor'] = column_name_split[0]
            new_df['property'] = '_'.join(column_name_split[:2])
            new_df['feature'] = '_'.join(column_name_split[2:])
            if column_name_split[0] == "mag":
                new_df['platform'] = "AK8963"
            else:
                new_df['platform'] = 'MPU-6500'

        new_df['timestamp_gps'] = PVS_Dataset['timestamp_gps'].to_numpy()
        new_df['vehicle'] = PVS_Dataset['Vehicle'].to_numpy()
        new_df['procedure'] = PVS_Dataset['procedure'].to_numpy()
        new_df['driver'] = PVS_Dataset['Driver'].to_numpy()
        new_df['scenario'] = PVS_Dataset['Scenario'].to_numpy()
        new_df['latitude'] = PVS_Dataset['latitude'].to_numpy()
        new_df['longitude'] = PVS_Dataset['longitude'].to_numpy()
        new_df['Distance'] = PVS_Dataset['Scenario'].to_numpy()
        new_df['datetime'] = PVS_Dataset['timestamp'].to_numpy()
        new_df['Road_Surface_Type_Labels'] = PVS_Dataset['Road_Surface_Type_Labels'].to_numpy()
        new_df['Road_Surface_Condition'] = PVS_Dataset['Road_Surface_Condition'].to_numpy()
        new_df['Road_Roughness_Condition_Left'] = PVS_Dataset['Road_Roughness_Condition_Left'].to_numpy()
        new_df['Road_Roughness_Condition_Right'] = PVS_Dataset['Road_Roughness_Condition_Right'].to_numpy()
        new_df['Speed_Bump_Types'] = PVS_Dataset['Speed_Bump_Types'].to_numpy()
        table_list.append(new_df)

features_table = pd.concat(table_list, axis=0)
features_table['unique_id'] = [str(uuid.uuid4())[:8] for _ in range(len(features_table))]
features_table.to_csv('./outputs/rowData/features.csv')
print(features_table.columns)

Index(['measure', 'sensor', 'property', 'feature', 'platform', 'timestamp_gps',
       'vehicle', 'procedure', 'driver', 'scenario', 'latitude', 'longitude',
       'Distance', 'datetime', 'Road_Surface_Type_Labels',
       'Road_Surface_Condition', 'Road_Roughness_Condition_Left',
       'Road_Roughness_Condition_Right', 'Speed_Bump_Types', 'unique_id'],
      dtype='object')


In [12]:
import morph_kgc
graph = morph_kgc.materialize("./mapping/config.ini")
graph.serialize(destination="./outputs/triples/result.txt")

INFO | 2023-06-07 15:22:05,772 | Parallelization is not supported for darwin when running as a library. If you need to speed up your data integration pipeline, please run through the command line.
INFO | 2023-06-07 15:22:06,197 | 60 mapping rules retrieved.
INFO | 2023-06-07 15:22:06,261 | Mapping partition with 60 groups generated.
INFO | 2023-06-07 15:22:06,263 | Maximum number of rules within mapping group: 1.
INFO | 2023-06-07 15:22:06,263 | Mappings processed in 0.480 seconds.
INFO | 2023-06-07 15:22:10,737 | Number of triples generated in total: 584126.


<Graph identifier=N2d430c67def54c6f80326cc46573e4a4 (<class 'rdflib.graph.Graph'>)>

In [13]:
from rdflib import Graph, store

g = Graph(store='neo4j-cypher')
theconfig = {'uri': "bolt://localhost:7687", 'database': 'neo4j', 'auth': {'user': "neo4j", 'pwd': "28071382"}}

g.open(theconfig, create = False)

Uniqueness constraint on :Resource(uri) found. 


In [9]:
g.store.startBatchedWrite()
g.parse("./outputs/triples/result.txt", format="ttl")
g.store.endBatchedWrite()

INFO | 2023-06-05 06:59:33,455 | starting import. Batch size 10000
INFO | 2023-06-05 07:03:30,429 | batch import done


In [14]:
from neo4j import GraphDatabase

url = 'bolt://localhost:7687'
user = 'neo4j'
password = '28071382'

driver = GraphDatabase.driver(url, auth=(user, password))

cypher_queries_node_features = [
    "MATCH (n:ObservableProperty) RETURN n.value AS ObservableProperty",
    "MATCH (n:Sensor) RETURN n.value AS Sensor",
    "MATCH (n:FeatureOfInterest) RETURN n.value AS FeatureOfInterest",
    "MATCH (n:Platform) RETURN n.value AS Platform",
    "MATCH (n:vehicle) RETURN n.value AS vehicle",
    "MATCH (n:procedure) RETURN n.value AS procedure",
    "MATCH (n:Scenario) RETURN n.value AS Scenario",
    "MATCH (n:Result) RETURN n.value AS Result",
    "MATCH (n:Point) RETURN n.lat AS lat, n.lat AS long",
    "MATCH (n:Observation) RETURN n.hasSimpleResult AS simpleResult",
    "MATCH (n:Driver) RETURN n.value AS Driver",
    "Match (n:Observation) RETURN n.roadTypeLabel as Labels"
]

results = []
with driver.session() as session:
    for query in cypher_queries_node_features:
        result = session.run(query)
        records = list(result)
        results.append(records)
# Create a DataFrame from the query results
ObservableProperty = pd.DataFrame({
    "ObservableProperty": [record["ObservableProperty"] for record in results[0]],
})
Sensor = pd.DataFrame({
    "Sensor": [record["Sensor"] for record in results[1]],
})
FeatureOfInterest = pd.DataFrame({
    "FeatureOfInterest": [record["FeatureOfInterest"] for record in results[2]],
})
Platform = pd.DataFrame({
    "Platform": [record["Platform"] for record in results[3]],
})
vehicle = pd.DataFrame({
    "vehicle": [record["vehicle"] for record in results[4]],
})
procedure = pd.DataFrame({
    "procedure": [record["procedure"] for record in results[5]],
})
Scenario = pd.DataFrame({
    "Scenario": [record["Scenario"] for record in results[6]],
})
Result = pd.DataFrame({
    "Result": [float(record["Result"]) for record in results[7]],
})
Point = pd.DataFrame({
    "lat": [float(record["lat"]) for record in results[8]],
     "long": [float(record["long"]) for record in results[8]]
})
Observation = pd.DataFrame({
    "Observation": [float(record["simpleResult"]) for record in results[9]],
})
Driver = pd.DataFrame({
    "Driver": [record["Driver"] for record in results[10]],
})

Labels = pd.DataFrame({
    "Labels": [record["Labels"] for record in results[11]],
})

FeatureOfInterest = pd.get_dummies(FeatureOfInterest["FeatureOfInterest"])
ObservableProperty = pd.get_dummies(ObservableProperty["ObservableProperty"])
Sensor = pd.get_dummies(Sensor["Sensor"])
Platform = pd.get_dummies(Platform["Platform"])
vehicle = pd.get_dummies(vehicle["vehicle"])
procedure = pd.get_dummies(procedure["procedure"])
Scenario = pd.get_dummies(Scenario["Scenario"])
Driver = pd.get_dummies(Driver["Driver"])

Labels['Labels'] = pd.factorize(Labels['Labels'])[0]

dict_of_node_features = dict(Result=Result, Point=Point,Observation=Observation,
                             FeatureOfInterest=FeatureOfInterest, ObservableProperty=ObservableProperty,
                             Sensor=Sensor, Platform=Platform, Vehicle=vehicle,
                             Procedure=procedure,Scenario=Scenario, Driver=Driver)
print(Labels)

driver.close()

       Labels
0           0
1           1
2           1
3           1
4           2
...       ...
26995       0
26996       1
26997       2
26998       2
26999       1

[27000 rows x 1 columns]


In [15]:
import torch
print(type(FeatureOfInterest.values))

# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).long().to(device)

<class 'numpy.ndarray'>


In [16]:
cypher_queries_edge_list = [
    "MATCH (s)-[r:followed]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:hasDrivenWith]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:hasFeatureOfInterest]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:hasPart]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:hasResult]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:hosts]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:location]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:madeObservation]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:observedProperty]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:observes]-(o) RETURN ID(s) as domain , ID(o) as range",
    "MATCH (s)-[r:usedProcedure]-(o) RETURN ID(s) as domain , ID(o) as range"
]

all_relations = []
with driver.session() as session:
    for query in cypher_queries_edge_list:
        result = session.run(query)
        records = list(result)
        all_relations.append(records)

followed = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[0]],
     "range": [record["range"] for record in all_relations[0]]
}).T
hasDrivenWith = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[1]],
     "range": [record["range"] for record in all_relations[1]]
}).T
hasFeatureOfInterest = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[2]],
     "range": [record["range"] for record in all_relations[2]]
}).T
hasPart = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[3]],
     "range": [record["range"] for record in all_relations[3]]
}).T
hasResult = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[4]],
     "range": [record["range"] for record in all_relations[4]]
}).T
hosts = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[5]],
     "range": [record["range"] for record in all_relations[5]]
}).T
location = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[6]],
     "range": [record["range"] for record in all_relations[6]]
}).T
madeObservation = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[7]],
     "range": [record["range"] for record in all_relations[7]]
}).T
observedProperty = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[8]],
     "range": [record["range"] for record in all_relations[8]]
}).T
observes = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[9]],
     "range": [record["range"] for record in all_relations[9]]
}).T
usedProcedure = pd.DataFrame({
    "domain": [record["domain"] for record in all_relations[10]],
     "range": [record["range"] for record in all_relations[10]]
}).T

dict_of_edges = dict(Driver_Followed_Scenario=followed, Driver_hasDrivenWith_Vehicle=hasDrivenWith,
                     Observation_hasFeatureOfInterest_FeatureOfInterest=hasFeatureOfInterest,
                     Vehicle_hasPart_FeatureOfInterest=hasPart,Observation_hasResult_Result=hasResult,
                     Sensor_hosts_Platform=hosts, Observation_Location_Point=location,
                     Sensor_madeObservation_Observation=madeObservation,
                     Sensor_observedProperty_ObservableProperty=observedProperty,
                     Sensor_Observes_ObservableProperty=observes, Observation_usedProcedure_Procedure=usedProcedure)


driver.close()

In [17]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data = HeteroData()
for node in dict_of_node_features :
    tensor_node = df_to_tensor(dict_of_node_features[node])
    data[node].x = tensor_node
for relation in dict_of_edges:
    tensor_edge = df_to_tensor(dict_of_edges[relation])
    data[relation.split("_")[0],relation.split("_")[1],relation.split("_")[2]].edge_index = tensor_edge

Label = torch.from_numpy(Labels.values).float()
data['Observation'].y = Label
homogeneous_data = data.to_homogeneous()
print(homogeneous_data)
transformer = T.RandomNodeSplit()
data = transformer(data)
print(data)
#print(homogeneous_data.metadata)

Data(edge_index=[2, 324070], y=[50373, 1], node_type=[50373], edge_type=[324070])
HeteroData(
  Result={ x=[22405, 1] },
  Point={ x=[938, 2] },
  Observation={
    x=[27000, 1],
    y=[27000, 1],
    train_mask=[27000],
    val_mask=[27000],
    test_mask=[27000]
  },
  FeatureOfInterest={ x=[3, 3] },
  ObservableProperty={ x=[10, 10] },
  Sensor={ x=[4, 4] },
  Platform={ x=[2, 2] },
  Vehicle={ x=[3, 3] },
  Procedure={ x=[2, 2] },
  Scenario={ x=[3, 3] },
  Driver={ x=[3, 3] },
  (Driver, Followed, Scenario)={ edge_index=[2, 18] },
  (Driver, hasDrivenWith, Vehicle)={ edge_index=[2, 6] },
  (Observation, hasFeatureOfInterest, FeatureOfInterest)={ edge_index=[2, 54000] },
  (Vehicle, hasPart, FeatureOfInterest)={ edge_index=[2, 18] },
  (Observation, hasResult, Result)={ edge_index=[2, 54000] },
  (Sensor, hosts, Platform)={ edge_index=[2, 8] },
  (Observation, Location, Point)={ edge_index=[2, 54000] },
  (Sensor, madeObservation, Observation)={ edge_index=[2, 54000] },
  (Sensor, 

In [18]:
from torch_geometric.nn import GATConv, Linear, to_hetero

data = T.ToUndirected()(data)
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False)
        self.lin1 = Linear(-1, hidden_channels)
        self.conv2 = GATConv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = x.relu()
        x = self.conv2(x, edge_index) + self.lin2(x)
        return x


model = GAT(hidden_channels=64, out_channels=3)
model = to_hetero(model, data.metadata(), aggr='sum')
print(model)

GraphModule(
  (conv1): ModuleDict(
    (Driver__Followed__Scenario): GATConv((-1, -1), 64, heads=1)
    (Driver__hasDrivenWith__Vehicle): GATConv((-1, -1), 64, heads=1)
    (Observation__hasFeatureOfInterest__FeatureOfInterest): GATConv((-1, -1), 64, heads=1)
    (Vehicle__hasPart__FeatureOfInterest): GATConv((-1, -1), 64, heads=1)
    (Observation__hasResult__Result): GATConv((-1, -1), 64, heads=1)
    (Sensor__hosts__Platform): GATConv((-1, -1), 64, heads=1)
    (Observation__Location__Point): GATConv((-1, -1), 64, heads=1)
    (Sensor__madeObservation__Observation): GATConv((-1, -1), 64, heads=1)
    (Sensor__observedProperty__ObservableProperty): GATConv((-1, -1), 64, heads=1)
    (Sensor__Observes__ObservableProperty): GATConv((-1, -1), 64, heads=1)
    (Observation__usedProcedure__Procedure): GATConv((-1, -1), 64, heads=1)
    (Scenario__rev_Followed__Driver): GATConv((-1, -1), 64, heads=1)
    (Vehicle__rev_hasDrivenWith__Driver): GATConv((-1, -1), 64, heads=1)
    (FeatureOfIn

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import NeighborLoader

transform = T.ToUndirected()  # Add reverse edge types.

train_loader = NeighborLoader(
    data,
    # Sample 15 neighbors for each node and each edge type for 2 iterations:
    num_neighbors=[5] * 2,
    # Use a batch size of 128 for sampling training nodes of type "paper":
    batch_size=128,
    input_nodes=('Observation', data['Observation'].train_mask),
)

batch = next(iter(train_loader))

In [206]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv , to_hetero #GATConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # Initialize the layers
        self.conv1 = GCNConv((-1, -1), hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.out = Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        # First Message Passing Layer (Transformation)
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        # Second Message Passing Layer
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        # Output layer
        x = F.softmax(self.out(x), dim=1)
        return

model = GCN(hidden_channels=64, out_channels=3)
model = to_hetero(model, data.metadata(), aggr='sum')
print(model)

TypeError: '>' not supported between instances of 'tuple' and 'int'